# Performance Analysis and Wrangling

## Gathering the Necessary Materials

### Dependencies

In [1]:
#importing the needed dependencies
import pandas as pd

### CSV's

In [2]:
#import the csvs
onemin_df = pd.read_csv('../CSVs/scraped/1min.csv')
fourmin_df = pd.read_csv('../CSVs/scraped/4min.csv')
thirtymin_df = pd.read_csv('../CSVs/scraped/30min.csv')
sixtymin_df = pd.read_csv('../CSVs/scraped/60min.csv')
onehun_df = pd.read_csv('../CSVs/scraped/100m.csv')
fivehun_df = pd.read_csv('../CSVs/scraped/500m.csv')
onek_df = pd.read_csv('../CSVs/scraped/1000m.csv')
twok_df = pd.read_csv('../CSVs/scraped/2000m.csv')
fivek_df = pd.read_csv('../CSVs/scraped/5000m.csv')
sixk_df = pd.read_csv('../CSVs/scraped/6000m.csv')
tenk_df = pd.read_csv('../CSVs/scraped/10000m.csv')
halfmarathon_df = pd.read_csv('../CSVs/scraped/HalfMarathon.csv')
marathon_df = pd.read_csv('../CSVs/scraped/Marathon.csv')

### Definging functions

In [3]:
#function to covert the imported time to seconds
def mintosec (time):
    #split the given along the : to get a list of hours, mins, secs
    split = time.split(':')
    
    #the if statement for correct calculation for either hh:mm:ss or mm:ss
    if len(split) <= 2:
        
        #mm:ss converson
        return ((int(split[0])*60)+(float(split[1])))
    else:
        
        #hh:mm:ss
        return (((int(split[0])*60)*60)+(int(split[1])*60)+(float(split[1])))

In [4]:
#the function to clean the for distance tests
def clean_forDistance (df, timeinsec, testname):
    
    #adding a column with the test names
    df['Test'] = testname
    
    #removing all of the rows that have a distance of zero
    df = df[df['Distance'] != 0]
    
    #find the split time with the C2 formula
    df['Split'] = df['Distance'].apply(lambda x: round(500*(timeinsec/x),2))
    
    #find the watts with the C2 formula
    df['Watts'] = df['Split'].apply(lambda x: round(2.8/(x/500)**3,2))
    
    #sort the data frame by the distance, highest values at the top
    df = df.sort_values(by = ['Distance'], ascending=False)
    
    #reset the index
    df = df.reset_index()
    
    #create a unique table id that will be used as the primary key in the SQL Database
    df['TestID'] = df["Test"] + df["index"].astype(str)
    
    #drop all of the unneeded colums
    df = df.drop(columns=['Unnamed: 0', 'Pos.', 'Type', 'index'])
    
    #drop all of the rows that do not have a verified distance
    df = df[df['Verified'] != 'No']
    
    #send the dataframe to CSV with out an index
    df.to_csv(f'../CSVs/cleaned/{testname}.csv', index = False)
    
    #retutn the cleaned dataframe
    return df

In [5]:
#the function to clean the for time tests
def clean_forTime (df, distance, testname):
    #adding a column with the test name
    df['Test'] = testname
    
    #drop any rows that don't have a value in time
    df = df.dropna(subset=['Time'])
    
    #apply the mintocec function to the time column
    df['Seconds'] = df['Time'].apply(mintosec)
    
    #find the split time with the C2 formula
    df['Split'] = df['Seconds'].apply(lambda x: round(500*(x/distance), 2))
    
    #find the watts with the C2 formula
    df['Watts'] = df['Split'].apply(lambda x: round(2.8/(x/500)**3,2))
    
    #sort the dataframe by watts, highest value at the top
    df.sort_values(by = ['Watts'], ascending=False)
    
    #reset the index
    df = df.reset_index()
    
    #create a unique table id that will be used as the primary key in the SQL Database
    df['TestID'] = df["Test"] + df["index"].astype(str)
    
    #drop any unnecessary columns
    df = df.drop(columns=['Unnamed: 0', 'Pos.', 'Type', 'index'])
    
    #drop all of the rows that do not have a verified distance
    df = df[df['Verified'] != 'No']
    
    #send the dataframe to CSV with out an index
    df.to_csv(f'../CSVs/cleaned/{testname}.csv', index = False)
    
    #return the dataframe
    return df

## Getting Clean

- running the functions on each of the dataframes
- printing the head to make sure it worked

In [6]:
onemin_clean = clean_forDistance(onemin_df, 60, '1min')
onemin_clean.head()

,Name,Age,Location,Country,Club/Affiliation,Distance,Verified,Season,Test,Split,Watts,TestID
0,Phil Clapp,29,NaN,GBR,NaN,431,Yes,2020,1min,69.61,1037.66,1min8161
1,Loren Howard,34,NaN,USA,NaN,429,Yes,2018,1min,69.93,1023.48,1min3421
2,Phil Clapp,28,NaN,GBR,NaN,428,Yes,2019,1min,70.09,1016.48,1min5618
3,Ross Love,33,"Friendswood, TX",USA,NaN,428,Yes,2018,1min,70.09,1016.48,1min3422
4,Ross Love,31,"Friendswood, TX",USA,NaN,422,Yes,2016,1min,71.09,974.19,1min0


In [7]:
onemin_clean.Verified.unique()

array(['Yes', 'Race'], dtype=object)

In [8]:
fourmin_clean = clean_forDistance(fourmin_df, 240, '4min')
fourmin_clean.head()

<ipython-input-4-575b3f812f86>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Split'] = df['Distance'].apply(lambda x: round(500*(timeinsec/x),2))
<ipython-input-4-575b3f812f86>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Watts'] = df['Split'].apply(lambda x: round(2.8/(x/500)**3,2))


,Name,Age,Location,Country,Club/Affiliation,Distance,Verified,Season,Test,Split,Watts,TestID
5,Andy Ward,34,Leyland,GBR,NaN,1457,Yes,2008,4min,82.36,626.50,4min4916
6,Olli Zeidler,21,Munich,GER,MAD Team IRC,1445,Yes,2018,4min,83.04,611.23,4min19943
8,Cameron Webster,25,NaN,NZL,NaN,1416,Yes,2020,4min,84.75,574.97,4min22872
9,Hamish Bond,34,NaN,NZL,NaN,1412,Yes,2020,4min,84.99,570.12,4min22873
10,Isaiah Harrison,16,"Coeur d'Alene, ID",USA,SUB-7 IRC,1411,Yes,2021,4min,85.05,568.91,4min25274


In [9]:
thirtymin_clean = clean_forDistance(thirtymin_df, 1800, '30min')
thirtymin_clean.head()

<ipython-input-4-575b3f812f86>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Split'] = df['Distance'].apply(lambda x: round(500*(timeinsec/x),2))
<ipython-input-4-575b3f812f86>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Watts'] = df['Split'].apply(lambda x: round(2.8/(x/500)**3,2))


,Name,Age,Location,Country,Club/Affiliation,Distance,Verified,Season,Test,Split,Watts,TestID
5,Olli Zeidler,21,Munich,GER,MAD Team IRC,9533,Yes,2018,30min,94.41,415.92,30min83945
7,Hamish Bond,34,NaN,NZL,NaN,9449,Yes,2020,30min,95.25,405.02,30min99135
11,Benjamin Reuter,30,Kiel,GER,NaN,9320,Race,2018,30min,96.57,388.63,30min83946
13,Benjamin Reuter,32,NaN,GER,SG Athletico Budelsdorf,9246,Race,2020,30min,97.34,379.48,30min99136
14,Benjamin Reuter,31,NaN,GER,SG Athletico Büdelsdorf,9221,Race,2019,30min,97.60,376.46,30min91345


In [10]:
sixtymin_clean = clean_forDistance(sixtymin_df, 3600, '60min')
sixtymin_clean.head()

<ipython-input-4-575b3f812f86>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Split'] = df['Distance'].apply(lambda x: round(500*(timeinsec/x),2))
<ipython-input-4-575b3f812f86>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Watts'] = df['Split'].apply(lambda x: round(2.8/(x/500)**3,2))


,Name,Age,Location,Country,Club/Affiliation,Distance,Verified,Season,Test,Split,Watts,TestID
0,Eric Murray,29,"Cambridge, New Zealand",NZL,NaN,18728,Yes,2012,60min,96.11,394.24,60min25505
2,Benjamin Reuter,30,Kiel,GER,NaN,18283,Race,2018,60min,98.45,366.79,60min42306
3,George Bridgewater,31,NaN,NZL,NaN,18259,Yes,2015,60min,98.58,365.34,60min34009
4,Graham Benton,34,London,GBR,MAD Team IRC,18221,Yes,2008,60min,98.79,363.02,60min13785
5,Mahe Drysdale,27,Cambridge,NZL,NaN,18128,Yes,2006,60min,99.29,357.56,60min8020


In [11]:
onehun_clean = clean_forTime(onehun_df, 100, '100m')
onehun_clean.head()

,Name,Age,Location,Country,Club/Affiliation,Time,Verified,Season,Test,Seconds,Split,Watts,TestID
0,Ross Love,30,"Friendswood, TX",USA,NaN,0:13.3,Yes,2016,100m,13.3,66.5,1190.15,100m0
1,noel Carey,40,glanmire,IRL,NaN,0:13.6,Yes,2016,100m,13.6,68.0,1113.12,100m1
2,Mike Reilly,24,"Varsity House, NJ",USA,NaN,0:13.7,Yes,2016,100m,13.7,68.5,1088.92,100m2
3,Jack Anderson,28,"St. Louis, MO",USA,"Project Deliverance, St. Louis, Missouri",0:13.8,Yes,2016,100m,13.8,69.0,1065.42,100m3
7,David Buer,35,"Atlanta, GA",USA,NaN,0:14.0,Yes,2016,100m,14.0,70.0,1020.41,100m7


In [12]:
fivehun_clean = clean_forTime(fivehun_df, 500, '500m')
fivehun_clean.head()

,Name,Age,Location,Country,Club/Affiliation,Time,Verified,Season,Test,Seconds,Split,Watts,TestID
3,John Stenard,43,"St. Petersburg, FL",USA,Massachusetts Institute of Technology,1:19.1,Race,2002,500m,79.1,79.1,707.19,500m3
18,Jim Hughes,29,"Shrewsbury, MA",USA,NaN,1:21.1,Race,2002,500m,81.1,81.1,656.15,500m18
101,Dominick Caste,40,"Orlando, FL",USA,NaN,1:25.8,Race,2002,500m,85.8,85.8,554.12,500m101
115,Taylor Levick,16,"Tampa, FL",USA,NaN,1:26.1,Race,2002,500m,86.1,86.1,548.35,500m115
135,Ransley Ilario,23,"Boca Raton, FL",USA,NaN,1:27.0,Race,2002,500m,87.0,87.0,531.51,500m135


In [13]:
onek_clean = clean_forTime(onek_df, 1000, '1000m')
onek_clean.head()

,Name,Age,Location,Country,Club/Affiliation,Time,Verified,Season,Test,Seconds,Split,Watts,TestID
227,Georges Devos,13,NaN,USA,NaN,3:15.8,Race,2002,1000m,195.8,97.9,373.01,1000m227
441,Ruben De Gendt,13,NaN,USA,NaN,3:24.6,Race,2002,1000m,204.6,102.3,326.92,1000m441
498,Joachim De Pauw,13,NaN,USA,NaN,3:26.6,Race,2002,1000m,206.6,103.3,317.52,1000m498
515,Bert Guenter,13,NaN,USA,NaN,3:27.2,Race,2002,1000m,207.2,103.6,314.77,1000m515
679,Mark Willman,16,"Tallahassee, FL",USA,NaN,3:33.8,Race,2002,1000m,213.8,106.9,286.51,1000m679


In [14]:
twok_df = twok_df.rename(columns={"Year": "Season"})
twok_clean = clean_forTime(twok_df, 2000, '2000m')
twok_clean.head()

,Name,Age,Location,Country,Club/Affiliation,Time,Verified,Season,Test,Seconds,Split,Watts,TestID
0,Matthias Siejkowski,37,NaN,GER,NaN,5:37.0,Race,2002,2000m,337.0,84.25,585.27,2000m0
2,Matthew Pinsent,31,NaN,GBR,Leander Club,5:47.5,Race,2002,2000m,347.5,86.88,533.71,2000m2
3,James Cracknell,29,NaN,GBR,Leander Club,5:47.6,Race,2002,2000m,347.6,86.90,533.35,2000m3
4,Chris Grothkopp,22,"Princeton, NJ",USA,NaN,5:49.4,Race,2002,2000m,349.4,87.35,525.15,2000m4
5,Jamie Schroeder,22,"Stanford, CA",USA,NaN,5:50.3,Race,2002,2000m,350.3,87.58,521.02,2000m5


In [15]:
fivek_clean = clean_forTime(fivek_df, 5000, '5000m')
fivek_clean.head()
# fivek_df.head()

<ipython-input-5-1aad651ee563>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Seconds'] = df['Time'].apply(mintosec)
<ipython-input-5-1aad651ee563>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Split'] = df['Seconds'].apply(lambda x: round(500*(x/distance), 2))
<ipython-input-5-1aad651ee563>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

,Name,Age,Location,Country,Club/Affiliation,Time,Verified,Season,Test,Seconds,Split,Watts,TestID
4052,Anna Bailey,52,Worcester,GBR,NaN,18:47.2,Race,2003,5000m,1127.2,112.72,244.38,5000m4052
17193,Carlos Lousto,44,NY,USA,Rochester Institute of Technology,21:27.9,Yes,2005,5000m,1287.9,128.79,163.84,5000m17193
19640,Matthew Kavanagh,29,Auckland,NZL,NaN,15:58.2,Yes,2006,5000m,958.2,95.82,397.83,5000m19640
19643,Peter Berg,23,Helsingborg,SWE,NaN,16:13.5,Yes,2006,5000m,973.5,97.35,379.37,5000m19643
19644,Dan Staite,32,Cambridge,GBR,NaN,16:18.4,Yes,2006,5000m,978.4,97.84,373.70,5000m19644


In [16]:
fivek_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180219 entries, 0 to 180218
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        180219 non-null  int64  
 1   Pos.              180218 non-null  float64
 2   Name              180198 non-null  object 
 3   Age               180219 non-null  int64  
 4   Location          152243 non-null  object 
 5   Country           180216 non-null  object 
 6   Club/Affiliation  58330 non-null   object 
 7   Time              180218 non-null  object 
 8   Type              180219 non-null  object 
 9   Verified          180219 non-null  object 
 10  Season            180219 non-null  int64  
 11  Test              180219 non-null  object 
dtypes: float64(1), int64(3), object(8)
memory usage: 16.5+ MB


In [17]:
sixk_clean = clean_forTime(sixk_df, 6000, '6000m')
sixk_clean.head()

,Name,Age,Location,Country,Club/Affiliation,Time,Verified,Season,Test,Seconds,Split,Watts,TestID
1957,Malcolm Fawcett,62,York,GBR,NaN,22:37.3,Yes,2003,6000m,1357.3,113.11,241.86,6000m1957
2017,Christopher Collins,70,"Charlottesville, VA",USA,NaN,22:48.1,Race,2003,6000m,1368.1,114.01,236.18,6000m2017
2136,Larry Eichelberger,58,"North Garden, VA",USA,NaN,23:08.4,Race,2003,6000m,1388.4,115.70,225.98,6000m2136
2755,Lise Rowe,43,"Charlottesville, VA",USA,"ACAC, Charlottesville, VA",25:09.4,Race,2003,6000m,1509.4,125.78,175.89,6000m2755
4548,Anna Bailey,52,Worcester,GBR,NaN,22:54.6,Race,2004,6000m,1374.6,114.55,232.85,6000m4548


In [18]:
tenk_clean = clean_forTime(tenk_df, 10000, '10000m')
tenk_clean.head()

,Name,Age,Location,Country,Club/Affiliation,Time,Verified,Season,Test,Seconds,Split,Watts,TestID
2561,Line Espedal,22,"Bloomington, IN",USA,Indiana University,35:55.0,Yes,2003,10000m,2155.0,107.75,279.78,10000m2561
15468,Andreas van Tonder,44,White River,RSA,Shosholoza,33:45.5,Yes,2006,10000m,2025.5,101.28,336.90,10000m15468
15471,Andrew Jones,23,west cross,GBR,Taff Attack Racing,33:55.5,Yes,2006,10000m,2035.5,101.78,331.96,10000m15471
15487,Mark Wilsmore,32,NaN,AUS,NaN,34:30.0,Race,2006,10000m,2070.0,103.50,315.68,10000m15487
15491,James Crawford,50,"Manhasset, NY",USA,Ever Green Boat Club,34:33.7,Yes,2006,10000m,2073.7,103.68,314.04,10000m15491


In [19]:
halfmarathon_clean = clean_forTime(halfmarathon_df, 21097, '21097m')
halfmarathon_clean.head()

,Name,Age,Location,Country,Club/Affiliation,Time,Verified,Season,Test,Seconds,Split,Watts,TestID
4051,Frédéric KOWAL,34,Nogent sur seine,FRA,NaN,1:13:24.7,Yes,2006,21097m,4393.0,104.11,310.16,21097m4051
4052,Diego Rivieri,28,Casalmaggiore,ITA,"Prosport Trento, Italy",1:13:36.8,Race,2006,21097m,4393.0,104.11,310.16,21097m4052
4053,Dan Staite,32,Cambridge,GBR,NaN,1:13:56.3,Yes,2006,21097m,4393.0,104.11,310.16,21097m4053
4055,Nick Rockliff,48,Brotherton,GBR,TEAM OARSOME IRC,1:14:25.9,Yes,2006,21097m,4454.0,105.56,297.56,21097m4055
4056,Phil Giles,31,"Wollongong, NSW",AUS,NaN,1:14:35.7,Yes,2006,21097m,4454.0,105.56,297.56,21097m4056


In [20]:
marathon_clean = clean_forTime(marathon_df, 42195, '42195m')
marathon_clean.head()

,Name,Age,Location,Country,Club/Affiliation,Time,Verified,Season,Test,Seconds,Split,Watts,TestID
808,Pentti Runko,47,Maaninka,FIN,NaN,2:32:25.3,Yes,2007,42195m,9152.0,108.45,274.40,42195m808
809,Peter van Vliet,19,"Old Saybrook, CT",USA,United States Naval Academy,2:32:43.7,Yes,2007,42195m,9152.0,108.45,274.40,42195m809
813,Len Fechter,31,"Pittsburgh, PA",USA,c2TweetCrew.com,2:36:47.5,Yes,2007,42195m,9396.0,111.34,253.58,42195m813
815,Nick Wakeford,29,"Leederville, WA",AUS,West Australian Rowing Club,2:38:09.7,Yes,2007,42195m,9518.0,112.79,243.92,42195m815
819,Alex Brown,54,Nottingham,GBR,Old Wrecks,2:41:07.3,Yes,2007,42195m,9701.0,114.95,230.43,42195m819
